---
title: Jugar en un equipo de élite
subtitle: Un análisis de los factores que determinan la inclusión de futbolistas en los equipos de élite mundial
format: clean-revealjs
html-math-method:
  method: mathjax
  url: "https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"
author:
  - name: Matías Deneken
    email: matdknu@gmail.com
  - name: Julián Brunet 
    email: julianbrunetsegovia@gmail.com
  - name: José Tomás Rodríguez
    email: josetomasrodriguez@gmail.com
---


## Introducción: Idea de trabajo 

El presente trabajo busca explorar cuáles son los factores que predicen pertenencer a un equipo de la élite mundial. En particular, nos centraremos [La Liga]{.fg style="--col: #e64173"}, [Ligue 1]{.fg style="--col: #e64173"}, [Bundesliga]{.fg style="--col: #e64173"} y [Premier League]{.fg style="--col: #e64173"}.


Debido a que estamos lidiando con una variable dependiente dicotómica (pertenecer o no a un equipo de élite) se ***propone*** utilizará un modelo de regresión logística con la siguiente especificación.

En particular, interesa trabajar con un modelo que posea la siguiente ecuación: 

\begin{align*}
\ln\left(\frac{\mathbb{P}(\text{Elite=1})}{1-\mathbb{P}(\text{Elite=1})}\right) &= \beta_{0} + \beta_{Edad_i} + \beta_{Piel_i} \\
&\quad + \beta_{Pos_i} + \beta_{Edad^{2}_i} + \beta_{Piel_i \times Pos_i} + e_{i}
\end{align*}



## Datos

Se utilizará una bse de datos que contempla todos los jugadores de las ligas mencionadas para la [temporada 2013]{.bg style="--col: #e64173"}. 


Para entender un equipo de elite mundial, nos centraremos en los equipos que estuvieron en las tres mejores posiciones de cada una de las ligas. 

| La Liga            | Ligue 1               | Bundesliga           | Premiere League     |
|--------------------|-----------------------|----------------------|---------------------|
| FC Barcelona       | Paris Saint Germain   | Borussia Dortmund    | Chelsea             |
| Real Madrid        | Olympique Lyon        | Bayern Munich        | Manchester City     |
| Atletico de Madrid | Olympique Marseille   | Bayer Leverkusen     | Manchester United   |


La data fue obtenida de:

Auspurg, K., & Brüderl, J. (2021). Has the Credibility of the Social Sciences Been Credibly Destroyed? Reanalyzing the “Many Analysts, One Data Set” Project. Socius, 7. https://doi.org/10.1177/23780231211024421


## Datos
### Selección de variables


- `player`: nombre del jugador

- `club`: equipo del jugador

- `leagueCountry`: país del equipo del jugador (England, Germany, France, Spain)

- `birthday`: fecha de nacimiento del jugador

- `position`: posición en la que juega el jugador

- `rater1`: evaluación del color de piel del jugador, evaluador 1 

- `rater2`: evaluación del color de piel del jugador, evaluador 2

## Datos
### Selección de variables

- `player`: nombre del jugador

- **`club`: equipo del jugador**

- **`leagueCountry`: país del equipo del jugador (England, Germany, France, Spain)**

- **`birthday`: fecha de nacimiento del jugador**

- `position`: posición en la que juega el jugador

- `rater1`: evaluación del color de piel del jugador, evaluador 1 

- `rater2`: evaluación del color de piel del jugador, evaluador 2



## Naturaleza de los datos


In [ ]:
#| echo: true
import pandas as pd
import numpy as np
import os

# Cargar los datos
data = pd.read_csv('bbdd/redcard_data.csv')
data.head

data['age'] = 2024 - pd.to_datetime(data['birthday']).dt.year

## Transformación de variables
#### **Equipo de elite**

In [ ]:
#| echo: true
# Crear la variable 'elite'
data['elite'] = np.where(data['club'].isin([
    'FC Barcelona', 'Real Madrid','Atlético Madrid',
    'Borussia Dortmund', 'Bayern München','Bayer Leverkusen',
    'Olympique Lyon','Paris Saint Germain','Olympique Marseille',
    'Chelsea FC', 'Manchester City','Manchester United'
]), 1, 0)
data['elite'] = data['elite'].map({1: 'Elite', 0: 'No-Elite'})
data['elite'] = data['elite'].astype(str)

#### **Rater de sesgo racial y NA**

In [ ]:
#| echo: true
data['rater_mean'] = data.apply(
    lambda row: row['rater2'] * 2 if pd.isna(row['rater1']) else 
                (row['rater1'] * 2 if pd.isna(row['rater2']) else 
                 (row['rater1'] + row['rater2']) / 2), 
    axis=1
)
# Eliminar filas con cualquier valor NaN
data = data.dropna()
# Mostrar las primeras filas de 'rater_mean' y 'elite'
print(data[['rater_mean', 'elite']].head(2))

## Exploración de datos


In [ ]:
elite_frequencies = data['elite'].value_counts()

# Mostrar los resultados
print(elite_frequencies)

## Datos
### Datos descriptivos contraste élite - no elite 


In [ ]:
#| echo: true 
#| eval: false
import pandas as pd
# Calcular la frecuencia de 'elite' dentro de los grupos 'rater_mean' y 'age'
stats_by_rater_age = data.groupby(['rater_mean', 'age', 'elite'])['elite'].value_counts(normalize=True).unstack().reset_index()

# Mostrar los resultados
print(stats_by_rater_age)

## Visualización de datos

### Gráfico 1

## Visualización de datos

### Gráfico 2


## Visualización de datos


### Gráfico 3



## Hipótesis
### Hipótesis etaria. 

$H0$ La edad no tiene un efecto significativo en la probabilidad de pertenecer a un equipo de élite. En otras palabras, la probabilidad de que un jugador pertenezca a un equipo de élite es independiente de su edad.
    
$HA$ La edad tiene un efecto significativo en la probabilidad de pertenecer a un equipo de élite, con jugadores más jóvenes teniendo más probabilidades de pertenecer a un equipo de élite, mientras que las probabilidades disminuyen a medida que los jugadores envejecen.


La formulación de las hipótesis se basa en la observación de que en el contexto deportivo, y particularmente en deportes de alta competición, la edad de los jugadores puede influir en su capacidad para integrarse en equipos de élite. Sin embargo, en la medida que envejecen ellos podrán ser transferidos dado su pérdida en la capacidad física. 


## Hipótesis
### Hipótesis racial

$H0$ No existe una diferencia significativa en la probabilidad de que personas afrodescendientes o latinas jueguen en equipos de élite en función de la posición que ocupan en el campo (banda o no banda).
    
    
$HA$ Las personas afrodescendientes o latinas tienen una mayor probabilidad de jugar en equipos de élite cuando ocupan posiciones en la banda en comparación con otras posiciones en el campo.
\end{itemize}

La hipótesis se basa en la observación de que en el fútbol y otros deportes de equipo, las características físicas y el estilo de juego de los jugadores pueden influir en la posición en la que son más efectivos y, por ende, en su probabilidad de ser seleccionados por equipos de élite. 


# Regresión




## Proyecciones

laksjdlasjda
dakldjalkjdsca
aslkdjañc

1. D
2. D
3. D


  - name: Matías Deneken
    email: matdknu@gmail.com
  - name: Julian Brunet
    email: julianbrunetsegovia@gmail.com
  - name: José Tomás Rodríguez
    email: josetomasrodriguez@gmail.com





 <div style="text-align: center;">Esta presentación fue hecha en </div>

 <div style="color: blue; text-align: center;">Quarto</div>

<div style="text-align: center;">
   
</div>